In [7]:
import json
import torch
# import openai
import pandas as pd
from tqdm import tqdm
import time
# from dotenv import load_dotenv
import os
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

In [8]:
def load_tweets(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        tweets = json.load(f)
    return tweets

In [9]:
# tweets = [
#     {
#         "tweet_count": 1,
#         "unique_id": "azwar6102",
#         "username": "Azwar Rozali",
#         "text": "Sebenarnya presiden Prabowo tidak perlu banyak bicara, yang dibutuhkan rakyat aksi nyata. Menaikkan PPN, menaikkan pajak kendaraan bermotor, menaikkan gaji honorer Rp 500ribu tidak perlu 100 hari. Kenapa kasus-kasus lain diabaikan, pengesahan RUU rampas aset presiden diam.",
#         "hashtags": [],
#         "created_at": "2025-01-01 03:26:45",
#         "retweet_count": 0,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 2,
#         "unique_id": "B_dzoel",
#         "username": "°Bang_Dzoel°",
#         "text": "Hadiah 100 hari kerja kabinet Merah-Putih @prabowo\n\n1. PPN 12%\n2. Daun kelor pengganti Susu\n3. Judol Peringkat Tertinggi Dunia\n4. Memaafkan Koruptor\n5.\n6.\n7.\n\nMenuju Indonesia Emas\nJayalah Indonesiaku\n\n#100HariKabinetOmon \n\n. https://t.co/AIvUpH6PSu",
#         "hashtags": [
#             "100HariKabinetOmon"
#         ],
#         "created_at": "2025-01-01 01:34:19",
#         "retweet_count": 12,
#         "favorite_count": 28,
#         "reply_count": 5,
#         "image_paths": [
#             "C:\\Users\\hariz\\projects\\tweet_scraping\\images\\media_2_0.jpg"
#         ]
#     },
#     {
#         "tweet_count": 3,
#         "unique_id": "RiskyDeswandi",
#         "username": "riskydeswandi28",
#         "text": "H-19, penilaian 100 Hari Kerja Pemerintahan Republik Indonesia Kabinet Merah Putih.\n#prabowo",
#         "hashtags": [
#             "prabowo"
#         ],
#         "created_at": "2025-01-01 00:17:07",
#         "retweet_count": 0,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 4,
#         "unique_id": "AlimyFikri",
#         "username": "Singgih",
#         "text": "@Dennysiregar7 Baru belum 100 hari sdh bosan sama Prabowo bang @Dennysiregar7 , apa minta diganti bang?",
#         "hashtags": [],
#         "created_at": "2025-01-01 13:15:48",
#         "retweet_count": 0,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 5,
#         "unique_id": "DitriRizki_D3",
#         "username": "dit_RI",
#         "text": "@Mdy_Asmara1701 Blunder Pak Prabowo kok bisa milih dia jadi menteri… nilai 11/100 lagi buat pak Prabowo menuju masa habis 100 hari kerja pertama rezim Prabowo…  mau menangis kok Indonesia dibuat main2 sama penguasa, pendukung &amp; oligarki…",
#         "hashtags": [],
#         "created_at": "2025-01-01 23:56:41",
#         "retweet_count": 0,
#         "favorite_count": 9,
#         "reply_count": 1,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 93,
#         "unique_id": "azwar6102",
#         "username": "Azwar Rozali",
#         "text": "Catat sudah berapa banyak kebohongan Prabowo sebelum 100 hari: gaji guru ditambah 2 juta kenyataan 500 ribu, tidak akan naikkan pajak nyatanya PPN 12%, pajak motor naik 66%, PPN naik hanya barang mewah kenyataan air mineral kena juga dst.",
#         "hashtags": [],
#         "created_at": "2025-01-06 00:33:10",
#         "retweet_count": 0,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": []
#     },
#     {
#         "tweet_count": 94,
#         "unique_id": "RadioElshinta",
#         "username": "Radio Elshinta",
#         "text": "Kurang dari 100 Hari Menjabat, Prabowo Tepati Janji Makan Bergizi Gratis Mulai Dinikmati di 26 Provinsi Hari Ini\n\nBoyolali - Kurang dari 100 hari menjabat, Presiden RI Prabowo Subianto menepati janjinya yaitu mulai menerapkan program Makan Bergizi Gratis untuk para pelajar di 190 titik yang tersebar di 26 provinsi pada hari ini secara serentak, Senin (6/1).\n\nSalah satunya di SDN 1 Donohudan, Boyolali, Jawa Tengah. Di sekolah ini terdapat 136 murid dari kelas 1 sampai 6 yang mulai menerima Makan Bergizi Gratis. Menu yang diberikan pada hari ini ialah nasi, daging, sayur, dan susu.\n\nKepala Sekolah SDN 1 Donohudan Sri Sunarti mengatakan penerapan program Makan Bergizi Gratis terealisasi dengan baik di sekolahnya.\n\nIa menerangkan makanan dari dapur umum didistribusikan ke sekolah sekolah sekitar jam 8 pagi.\n\n“Benar-benar kami rasakan sangat menunjang kebiasaan anak Indonesia hebat sehingga nanti kedepannya dapat mencetak generasi muda yang benar-benar unggul dan handal di segala bidang,” ujar Sri kepada wartawan, Senin.\n\n“Semoga kegiatan ini bisa berjalan dengan lancar sehingga kami semua dapat rasakan manfaat dari program Makan Bergizi Gratis,” lanjutnya.\n\nApri Dwi Prasetyo, salah satu guru di SD tersebut pun mengungkapkan bagaimana antusiasme para guru dan siswa menyambut penerapan Makan Bergizi Gratis yang dimulai hari ini.\n\n“Anak-anak sangat antusias sekali mengikuti  kegiatan ini, dimulai dari mengambil makanan secara mengantre, berbaris dengan rapi, kemudian membawa makanan tersebut ke dalam kelas. Di dalam kelas diawali dengan doa bersama terlebih dahulu, kemudian mereka menikmati makanan tersebut secara bersama-sama,” jelasnya.\n\nIa pun mengungkapkan rasa haru dan syukur karena program ini sudah berjalan dan dirasakan oleh para muridnya.\n\n“Terharu dan bersyukur karena anak-anak dapat makanan gratis bergizi ini tanpa biaya. Harapan saya program ini, dapat terus berkembang kemudian membawa kebaikan dan memperbaiki kualitas hidup di masa yang akan datang dari anak-anak tersebut,” kata Apri.\n\nSeperti diketahui, program Makan Bergizi Gratis ini sudah mulai berjalan di 26 provinsi di Indonesia dari Aceh, Bali, Sumatera Barat, Sumatera Utara, Kepulauan Riau, Riau, Lampung, Banten, Jawa Barat, D.K.I. Jakarta, Jawa Tengah, Jawa Timur, D.I. Yogyakarta, Gorontalo, Sulawesi Barat, Sulawesi Selatan, Sulawesi Tenggara,\nSulawesi Utara, Kalimantan Timur, Kalimantan Utara, Kalimantan Selatan, Maluku, Maluku Utara, Nusa Tenggara Timur, hingga Papua Barat dan Papua Selatan.\n\nDapur untuk Makan Bergizi Gratis ini dikelola oleh Satuan Pelayanan Pemenuhan Gizi (SPPG) yang ditunjuk langsung oleh Badan Gizi Nasional (BGN). SPPG merupakan merupakan unit pelaksana program Makan Bergizi Gratis yang bertugas memasok makanan untuk para penerima manfaat program. \n\nSPPG bekerja sama dengan seorang ahli gizi dan seorang akuntan untuk memastikan pengawasan ketat terhadap kualitas gizi dan kelancaran distribusi makanan.\n\nSelain memastikan kecukupan gizi dalam setiap porsi MBG, SPPG juga bertugas mengawasi standar kebersihan, pengelolaan gizi, dan pengolahan limbah di setiap Dapur MBG dengan ketat. Adapun untuk mendukung keberlanjutan, nampan penyajian dirancang menggunakan bahan stainless steel yang higienis dan dapat digunakan ulang. @prabowo",
#         "hashtags": [],
#         "created_at": "2025-01-06 05:37:17",
#         "retweet_count": 1,
#         "favorite_count": 0,
#         "reply_count": 0,
#         "image_paths": [
#             "C:\\Users\\hariz\\projects\\tweet_scraping\\images\\media_94_0.jpg",
#             "C:\\Users\\hariz\\projects\\tweet_scraping\\images\\media_94_1.jpg",
#             "C:\\Users\\hariz\\projects\\tweet_scraping\\images\\media_94_2.jpg"
#         ]
#     },
#     {
#         "tweet_count": 95,
#         "unique_id": "RonaJateng5",
#         "username": "Rona Jateng",
#         "text": "Kurang dari 100 Hari Menjabat, Prabowo Tepati Janji Makan Bergizi Gratis Mulai Dinikmati di 26 Provinsi Hari Ini https://t.co/hO7cnJmKMt",
#         "hashtags": [],
#         "created_at": "2025-01-06 08:16:30",
#         "retweet_count": 0,
#         "favorite_count": 1,
#         "reply_count": 0,
#         "image_paths": []
#     }
# ]

In [10]:
# ChatGPT openAI api
# def analyze_sentiment(text):
#     """
#     Analyze the sentiment of the given text using the OpenAI API.
#     The result is not good, for now I'll not use this.
#     """
#     try:
#         response = openai.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "system", "content": "You are a sentiment analyzer for Bahasa Indonesia. Classify the following text as POSITIVE, NEGATIVE, or NEUTRAL. Only respond with one of these three words."},
#                 {"role": "user", "content": text}
#             ],
#             temperature=0
#         )
#         return response.choices[0].message.content.strip()
#     except Exception as e:
#         print(f"Error analyzing sentiment: {e}")
#         time.sleep(20)  # Wait if we hit rate limits
#         return "NEUTRAL"  # Default fallback

In [11]:
# IndoRoBERTa-Base-Sentiment-Indonesian-Social-Media
# Load the base tokenizer
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# Load the base model
# base_model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

sentiment_model = AutoModelForSequenceClassification.from_pretrained(
    "yogie27/IndoRoBERTa-Base-Sentiment-Indonesian-Social-Media"
)

def analyze_sentiment(text):
    text = text.replace('\n', ' ').strip()
    # tokens = tokenizer.tokenize(text)
    # print("Tokens:", tokens)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_mapping[predicted_class]


In [ ]:
# Load tweets
tweets = load_tweets('tweets.json')

# Create a list to store results
results = []

# Process each tweet
for tweet in tqdm(tweets, desc="Analyzing tweets"):
    sentiment = analyze_sentiment(tweet['text'])
    tweet['sentiment'] = sentiment  # Add sentiment directly to tweet object
    time.sleep(0.5)

# Save updated tweets back to the original file
with open('tweets.json', 'w', encoding='utf-8') as f:
    json.dump(tweets, f, ensure_ascii=False, indent=4)

print("Tweets updated with sentiment analysis")

    

Analyzing tweets: 100%|██████████| 3696/3696 [37:02<00:00,  1.66it/s]


Tweets updated with sentiment analysis


: 